In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series =  0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
  series +=  0.2 * np.sin((time - offset2) * (freq2 * 10 + 10))
  series +=  0.1 * np.random.rand(batch_size, n_steps) * 0.5
  return series[..., np.newaxis].astype(np.float32)

print(generate_time_series(32, 5).shape)

In [ ]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [ ]:
# naive forecasting (last element in data as prediction)
# difficult to outperform

print(np.mean(keras.losses.mean_squared_error(y_val, X_val[:, -1])))

In [ ]:
# FFNN

model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=[50, 1]),
  keras.layers.Dense(1),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

In [ ]:
# Simple RNN

model = keras.models.Sequential([
  keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

In [ ]:
# Deep RNN

model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=False),
  keras.layers.Dense(1),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

In [ ]:
# Using same model to predict multiple steps
new_steps = 10
series = generate_time_series(1, n_steps + new_steps)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:n_steps + new_steps]
X = X_new
for step in range(new_steps):
  y_pred_one = model.predict(X[:, step:])[:, np.newaxis, :]
  X = np.concatenate((X, y_pred_one), axis=1)
y_pred = X[:, n_steps:]
print(y_pred)
print(tf.reduce_mean(keras.losses.MSE(Y_new, y_pred)))

In [ ]:
# Guessing 10 at a times
n_steps = 50
new_steps = 10
series = generate_time_series(10000, n_steps + new_steps)
X_train, y_train = series[:7000, :n_steps], series[:7000, -new_steps:]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -new_steps:]
X_test, y_test = series[9000:, :n_steps], series[9000:, -new_steps:]

print(series.shape)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

In [ ]:
model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=False),
  keras.layers.Dense(10),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

In [ ]:
print(tf.reduce_mean(keras.losses.MSE(y_test.reshape(len(y_test), len(y_test[0])), model.predict(X_test))))

In [ ]:
# Sequence to sequence data
example_count = 10000
prediction_length = 10
Y = np.empty((example_count, n_steps, prediction_length))
for step_ahead in range(1, prediction_length + 1):
  Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
y_train = Y[:7000]
y_val = Y[7000:9000]
y_test = Y[9000:]
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
# Custom metric for evaluating since only the final output matters.
# During training it helped to use all outputs.
def last_time_step_mse(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

# sequence to sequence model
model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
  keras.layers.SimpleRNN(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
# # NOT SURE WHY THIS DOESN'T WORK

# class LNSimpleRNNCell(keras.layers.Layer):
#   def __init__(self, units, activation='tanh', **kwargs):
#     super().__init__(**kwargs)
#     self.state_size = units
#     self.output_size = units
    
#     self.layer_norm = keras.layers.LayerNormalization()
#     self.activation = keras.activations.get(activation)
  
#   def build(self, batch_input_shape):
#     print(batch_input_shape)
#     self.simpleRNNCell = keras.layers.SimpleRNN(self.output_size, activation=None, input_shape=batch_input_shape)
#     super().build(batch_input_shape)
  
#   def call(self, inputs, states):
#     new_outputs, new_states = self.simpleRNNCell(inputs, states)
#     normalized_outputs = self.activation(self.layer_norm(new_outputs))
#     return normalized_outputs, [normalized_outputs]
    

In [ ]:
# # WON'T WORK BECAUSE OF ABOVE CELL
# model = keras.models.Sequential([
#   keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
#   keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
#   keras.layers.TimeDistributed(keras.layers.Dense(10))
# ])

# model.compile(
#   loss=keras.losses.MSE,
#   metrics=[last_time_step_mse],
#   optimizer=keras.optimizers.Adam()
# )

# model.fit(X_train, y_train, epochs=20)
# print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
model = keras.models.Sequential([
  keras.layers.LSTM(20, return_sequences=True),
  keras.layers.LSTM(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
model = keras.models.Sequential([
  keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid'),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train[:, 3::2], epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test[:, 3::2], model.predict(X_test))))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for dilation in [1, 2, 4, 8] * 2:
  model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding='causal', activation='relu', dilation_rate=dilation))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
print(y_test.shape, model.predict(X_test).shape)

In [ ]:
import tensorflow_datasets as tfds

dataset = tfds.load('imagenet_sketch', as_supervised=True)
dataset = dataset['test']

In [ ]:
train = dataset.take(40000)
val = dataset.skip(40000).take(5000)
test = dataset.skip(45000)
print(len(train), len(val), len(test))

In [ ]:
# Exercise 9

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

def dsprint(ds, num=1):
  # print(len(ds))
  for item in ds.take(num):
    print(item)

In [ ]:
class_map = {}
# create sketch rnn dataset
data_dir = os.path.join('../', 'data', 'quickdraw')
filenames = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
class_count = len(filenames)

for i in range(class_count):
  class_map[filenames[i]] = i

def load_data(filenames, batch_size=32, set_type='train'):
  np_list = [np.load(filename, encoding='latin1', allow_pickle=True)[set_type] for filename in filenames]
  ds_list = [tf.data.Dataset.from_generator(lambda: np_list[i], tf.int16).map(lambda X: (X, i)) for i in range(len(np_list))]
  ds = tf.data.Dataset.from_tensor_slices(ds_list).interleave(
    lambda ds: ds,
    cycle_length=len(ds_list),
    block_length=1,
  )
  ds = ds.shuffle(len(ds_list) * 10).map(lambda X, label: (tf.reverse(X, [1]), label)).padded_batch(batch_size, padded_shapes=([None, 3], ()))
  return ds.map(lambda X, label: (tf.reverse(X, [1]), label))

test_set = np.load('../data/quickdraw/ambulance.full.npz', encoding='latin1', allow_pickle=True)['train']
print(len(test_set))
for item in test_set:
  print(item)

In [ ]:
train = load_data(filenames, set_type='train')
val = load_data(filenames, set_type='valid')
test = load_data(filenames, set_type='test')

In [ ]:
traint = train.take(800).prefetch(1)
valt = val.take(100).prefetch(1)
testt = test.take(100).prefetch(1)

In [ ]:
import matplotlib.pyplot as plt

def draw_sketch(sketch, label=None):
    origin = np.array([[0., 0., 0.]])
    sketch = np.r_[origin, sketch]
    stroke_end_indices = np.argwhere(sketch[:, -1]==1.)[:, 0]
    coordinates = np.cumsum(sketch[:, :2], axis=0)
    strokes = np.split(coordinates, stroke_end_indices + 1)
    plt.plot(coordinates[:, 0], -coordinates[:, 1], "y:")
    for stroke in strokes:
        plt.plot(stroke[:, 0], -stroke[:, 1], ".-")
    plt.axis("off")

i = 10
for item in traint.take(1):
    sketch = item[0][i]
    file = filenames[item[1][i].numpy()]
    print(file)
    draw_sketch(sketch)

In [ ]:
# WHY WONT THIS WORK
# Exercise number 9
import tensorflow_addons as tfa

# model = keras.models.Sequential([
#   keras.layers.InputLayer(input_shape=[None, 3]),
#   # keras.layers.BatchNormalization(),
#   keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid'),
#   # keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid'),
#   # keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid'),
#   # keras.layers.LayerNormalization(),
#   # keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid'),
#   # keras.layers.LayerNormalization(),
#   keras.layers.LSTM(100, return_sequences=True),
#   # keras.layers.LayerNormalization(),
#   keras.layers.LSTM(100, return_sequences=True),
#   # keras.layers.LayerNormalization(),
#   keras.layers.LSTM(100, return_sequences=False),
#   # keras.layers.LayerNormalization(),
#   keras.layers.BatchNormalization(),
#   keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
#   keras.layers.BatchNormalization(),
#   # keras.layers.Dropout(0.5),
#   keras.layers.Dense(50, activation='relu', kernel_initializer='he_normal'),
#   keras.layers.BatchNormalization(),
#   # keras.layers.Dropout(0.5),
#   keras.layers.Dense(20, activation='relu', kernel_initializer='he_normal'),
#   keras.layers.BatchNormalization(),
#   # keras.layers.Dropout(0.5),
#   keras.layers.Dense(class_count, activation='softmax')
# ])

# model = keras.models.Sequential([
#   keras.layers.InputLayer(input_shape=[None, 3]),
#   keras.layers.Conv1D(filters=50, kernel_size=4, strides=2, padding='same'),
#   keras.layers.Conv1D(filters=50, kernel_size=4, strides=1, padding='same'),
#   keras.layers.Conv1D(filters=100, kernel_size=4, strides=2, padding='same'),
#   keras.layers.Conv1D(filters=100, kernel_size=4, strides=1, padding='same'),
#   keras.layers.Conv1D(filters=200, kernel_size=4, strides=2, padding='same'),
#   keras.layers.Conv1D(filters=200, kernel_size=4, strides=1, padding='same'),
#   keras.layers.GlobalAveragePooling1D(),
#   keras.layers.Flatten(),
#   keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
#   # keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
#   keras.layers.Dense(class_count, activation='softmax')
# ])

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 3]))
for dilation in [1, 2, 4, 8, 16] * 2:
  model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding='causal', activation='relu', dilation_rate=dilation))
# model.add(keras.layers.Conv1D(filters=class_count, kernel_size=1, activation='softmax'))
model.add(keras.layers.GlobalAveragePooling1D())
model.add((keras.layers.Dense(class_count, activation='softmax')))

model.compile(
  loss=keras.losses.sparse_categorical_crossentropy,
  metrics=[keras.metrics.sparse_categorical_accuracy],
  optimizer=keras.optimizers.Nadam()
  # optimizer=keras.optimizers.SGD(0.001, 0.9, True)
)

# print(tf.argmax(model.predict(batch), axis=1))

model.fit(traint, validation_data=valt, epochs=20, callbacks=[
  keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
  keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=2),
])

In [ ]:
# Exercise 10
from IPython.display import Audio

def notes_to_frequencies(notes):
  # Frequency doubles when you go up one octave; there are 12 semi-tones
  # per octave; Note A on octave 4 is 440 Hz, and it is note number 69.
  return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
  note_duration = 60 / tempo # the tempo is measured in beats per minutes
  # To reduce click sound at every beat, we round the frequencies to try to
  # get the samples close to zero at the end of each note.
  frequencies = np.round(note_duration * frequencies) / note_duration
  n_samples = int(note_duration * sample_rate)
  time = np.linspace(0, note_duration, n_samples)
  sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
  # Removing all notes with frequencies ≤ 9 Hz (includes note 0 = silence)
  sine_waves *= (frequencies > 9.).reshape(-1, 1)
  return sine_waves.reshape(-1)

def chords_to_samples(chords, tempo, sample_rate):
  freqs = notes_to_frequencies(chords)
  freqs = np.r_[freqs, freqs[-1:]] # make last note a bit longer
  merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                    for melody in freqs.T], axis=0)
  n_fade_out_samples = sample_rate * 60 // tempo # fade out last note
  fade_out = np.linspace(1., 0., n_fade_out_samples)**2
  merged[-n_fade_out_samples:] *= fade_out
  return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
  samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
  if filepath:
    from scipy.io import wavfile
    samples = (2**15 * samples).astype(np.int16)
    wavfile.write(filepath, sample_rate, samples)
    return display(Audio(filepath))
  else:
    return display(Audio(samples, rate=sample_rate))

from pandas import read_csv

def load_set(dir, batch_size=32):
  X = [os.path.join(dir, filename) for filename in os.listdir(dir)]
  X = [read_csv(path).values for path in X]
  # X = np.split(X, batch_size)
  max_len = max(*[len(song) for song in X])
  print(max_len)
  X = [np.pad(song, ((640 - len(song), 0), (0, 0))) for song in X]
  Y = [np.append(song[1:], [[0, 0, 0, 0]], axis=0) for song in X]
  return np.stack(X, axis=0).astype(np.float32), np.stack(Y, axis=0).astype(np.float32) / 87

X_train, Y_train = load_set('../data/jsb_chorales/train/')
X_val, Y_val = load_set('../data/jsb_chorales/valid/')
X_test, Y_test = load_set('../data/jsb_chorales/test/')

train = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(8)
val = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(8)
test = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
play_chords(X_train[1])

In [ ]:
def last_time_step_mse(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

# model = keras.models.Sequential()
# model.add(keras.layers.InputLayer(input_shape=[None, 4]))
# for dilation in [1, 2, 4, 8, 16, 32] * 4:
#   model.add(keras.layers.Conv1D(filters=100, kernel_size=2, padding='causal', activation='relu', dilation_rate=dilation))
# model.add(keras.layers.Conv1D(filters=4, kernel_size=1))

model = keras.models.Sequential([
  keras.layers.InputLayer(input_shape=[None, 4]),
  keras.layers.LSTM(100, return_sequences=True),
  keras.layers.LSTM(100, return_sequences=True),
  keras.layers.LSTM(100, return_sequences=False),
  # keras.layers.TimeDistributed(keras.layers.Dense(4)),
  keras.layers.Dense(4),
])

print(X_train.shape)
print(model.predict(X_train).shape)

model.compile(
  loss=keras.losses.MSE,
  # loss=last_time_step_mse,
  # metrics=[keras.metrics.MSE],
  metrics=[last_time_step_mse],
  # optimizer=keras.optimizers.Nadam()
  optimizer=keras.optimizers.SGD(0.035, 0.9, True)
)

# print(tf.argmax(model.predict(batch), axis=1))

model.fit(train, validation_data=val, epochs=160, callbacks=[
  keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True),
  keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
])

In [ ]:
# print(Y_test[0].shape)
# print(Y_test[0].reshape(1, *Y_test[0].shape).shape)
print(Y_test[0].reshape(1, *Y_test[0].shape))
print(model.predict(Y_test[0].reshape(1, *Y_test[0].shape)))